## Import libraries

In [6]:
try:
    from google.colab import drive

    IN_COLAB = True
    drive.mount('/content/drive')
    !pip install -qU python-dotenv scikit-learn xgboost==1.5.0
    %cd /content/drive/MyDrive/Best ML model ever/

except:
    IN_COLAB = False
    # Set working directory
    %cd ..
!pwd

/mnt/d/Google Drive/Best ML model ever
/mnt/d/Google Drive/Best ML model ever


In [52]:
import pandas as pd
import numpy as np
import json

from src.train_search_best_params import preprocess
from src import model_dispatcher
from collections import namedtuple
import xgboost

import warnings
warnings.filterwarnings("ignore") 


## Load the data

In [53]:
try:
    from google.colab import drive

    IN_COLAB = True
    drive.mount('/content/drive')
except:
    IN_COLAB = False

In [70]:
path = "input/drinking_water_potability.csv"

df = pd.read_csv(path)
df = df.sample(frac=1, random_state=1).reset_index(drop=True)
nb_tot_rows = df.shape[0]
df_features = df.drop("Potability", axis=1)
df

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,6.063355,160.767311,14775.14560,7.484104,305.828553,327.270239,12.309016,69.038454,3.467337,0
1,NaN,233.020134,27071.11862,6.220936,298.112645,357.119622,16.768945,51.284400,4.284879,0
2,9.252857,168.040751,50279.26243,5.905056,415.450810,400.003589,11.949854,62.256881,3.300586,0
3,3.989032,216.076138,40175.20609,7.487423,309.800796,399.047181,11.186739,86.957464,4.409414,0
4,9.454119,224.817132,21379.96393,5.407692,227.665635,431.613001,15.772334,52.033845,4.058626,1
...,...,...,...,...,...,...,...,...,...,...
3271,6.359431,208.203021,23347.17271,9.000395,NaN,336.585610,14.173906,NaN,3.636495,1
3272,NaN,192.202168,34160.92514,8.963156,363.472798,474.781734,15.905270,50.807825,2.998335,0
3273,NaN,142.145566,45141.68604,6.030640,240.198505,369.280429,20.605552,70.168389,4.604725,1
3274,4.814136,205.214041,17650.40505,8.121080,350.487939,414.030709,10.999416,47.402666,5.190852,0


# Loading best learners

In [55]:
pd.options.display.max_colwidth = 100
leaderboard = pd.read_csv('predictions/results_all_models.csv')
leaderboard = leaderboard.sort_values('f1_score', ascending=False)
leaderboard.head(10)

,model_name,preprocessing_params,model_params,accuracy,auc,f1_score
1862,xgb,"{""missing"": ""remove_rows"", ""scaling"": ""min_max"", ""add_Solids_log"": false, ""poly_degree"": 3}","{""alpha"": 0, ""lambda"": 0}",0.670588,0.690938,0.558360
1808,xgb,"{""missing"": ""remove_rows"", ""scaling"": ""standard"", ""add_Solids_log"": false, ""poly_degree"": 3}","{""alpha"": 0, ""lambda"": 0}",0.670588,0.690938,0.558360
3505,svm,"{""missing"": ""remove_rows"", ""scaling"": ""standard"", ""add_Solids_log"": true, ""poly_degree"": 2}","{""class_weight"": ""balanced"", ""kernel"": ""rbf""}",0.645201,0.678650,0.554005
3521,svm,"{""missing"": ""remove_rows"", ""scaling"": ""standard"", ""add_Solids_log"": false, ""poly_degree"": 1}","{""class_weight"": ""balanced"", ""kernel"": ""rbf""}",0.647059,0.684065,0.552465
3513,svm,"{""missing"": ""remove_rows"", ""scaling"": ""standard"", ""add_Solids_log"": true, ""poly_degree"": 3}","{""class_weight"": ""balanced"", ""kernel"": ""rbf""}",0.642724,0.676768,0.552388
3296,svm,"{""missing"": ""mean"", ""scaling"": ""min_max"", ""add_Solids_log"": false, ""poly_degree"": 3}","{""class_weight"": ""balanced"", ""kernel"": ""poly""}",0.554198,0.644483,0.551999
3584,svm,"{""missing"": ""remove_rows"", ""scaling"": ""min_max"", ""add_Solids_log"": false, ""poly_degree"": 3}","{""class_weight"": ""balanced"", ""kernel"": ""poly""}",0.647059,0.688336,0.551467
3537,svm,"{""missing"": ""remove_rows"", ""scaling"": ""standard"", ""add_Solids_log"": false, ""poly_degree"": 3}","{""class_weight"": ""balanced"", ""kernel"": ""rbf""}",0.641486,0.676180,0.551286
3576,svm,"{""missing"": ""remove_rows"", ""scaling"": ""min_max"", ""add_Solids_log"": false, ""poly_degree"": 2}","{""class_weight"": ""balanced"", ""kernel"": ""poly""}",0.650774,0.679562,0.551006
3824,svm,"{""missing"": ""knn"", ""scaling"": ""standard"", ""add_Solids_log"": false, ""poly_degree"": 3}","{""class_weight"": ""balanced"", ""kernel"": ""poly""}",0.502290,0.596897,0.551004


In [56]:
# take best performing model of each type excluding the "constant" model
best_models = leaderboard.drop_duplicates('model_name')[leaderboard['model_name']!='constant']
best_models

,model_name,preprocessing_params,model_params,accuracy,auc,f1_score
1862,xgb,"{""missing"": ""remove_rows"", ""scaling"": ""min_max"", ""add_Solids_log"": false, ""poly_degree"": 3}","{""alpha"": 0, ""lambda"": 0}",0.670588,0.690938,0.558360
3505,svm,"{""missing"": ""remove_rows"", ""scaling"": ""standard"", ""add_Solids_log"": true, ""poly_degree"": 2}","{""class_weight"": ""balanced"", ""kernel"": ""rbf""}",0.645201,0.678650,0.554005
616,dt_gini,"{""missing"": ""remove_rows"", ""scaling"": ""min_max"", ""add_Solids_log"": false, ""poly_degree"": 1}","{""class_weight"": null, ""splitter"": ""random""}",0.604334,0.596781,0.531293
1255,rf,"{""missing"": ""remove_rows"", ""scaling"": ""standard"", ""add_Solids_log"": true, ""poly_degree"": 3}","{""class_weight"": null, ""criterion"": ""gini""}",0.673065,0.704062,0.527397
951,dt_entropy,"{""missing"": ""remove_rows"", ""scaling"": ""min_max"", ""add_Solids_log"": false, ""poly_degree"": 2}","{""class_weight"": null, ""splitter"": ""best""}",0.609288,0.598242,0.527131
2695,log_reg,"{""missing"": ""remove_rows"", ""scaling"": ""standard"", ""add_Solids_log"": false, ""poly_degree"": 3}","{""penalty"": ""none"", ""solver"": ""lbfgs""}",0.670588,0.692910,0.518937
2384,extratrees,"{""missing"": ""remove_rows"", ""scaling"": ""min_max"", ""add_Solids_log"": false, ""poly_degree"": 3}","{""class_weight"": null, ""criterion"": ""entropy""}",0.676780,0.696226,0.513428
5078,knn,"{""missing"": ""remove_rows"", ""scaling"": ""standard"", ""add_Solids_log"": true, ""poly_degree"": 1}","{""n_neighbors"": 3, ""p"": 2, ""weights"": ""distance""}",0.620433,0.623580,0.493799
3022,svm_calib,"{""missing"": ""remove_rows"", ""scaling"": ""standard"", ""add_Solids_log"": true, ""poly_degree"": 3}","{""ensemble"": false, ""method"": ""isotonic""}",0.666254,0.683332,0.473013
4062,naive_bayes,"{""missing"": ""remove_rows"", ""scaling"": ""min_max"", ""add_Solids_log"": false, ""poly_degree"": 2}","{""var_smoothing"": 1e-05}",0.607430,0.600581,0.459473


In [12]:
Learner = namedtuple('Learner', ['pre_pipeline', 'model'])
learners = []
for ix, row in best_models.iterrows():
    model_name, preprocessing_params, model_params = row[['model_name', 'preprocessing_params', 'model_params']]
    preprocessing_params = json.loads(preprocessing_params)
    model_params = json.loads(model_params)

    # preprocess data
    print(f"\nPreprocessing... {preprocessing_params}")
    X = df.drop(["Potability", "kfold"], axis=1, errors='ignore').values
    y = df.Potability.values
    X, y, pre_pipeline = preprocess(X, y, model_name, preprocess_params=preprocessing_params, save=False, return_pipeline=True)
    print(f"Preprocessing results: X of shape {X.shape}")

    # load model
    model = model_dispatcher.models[model_name]["model"](**model_dispatcher.models[model_name]["base_model_params"])
    model.set_params(**model_params)
    print(model)

    # train model
    model.fit(X, y)

    # save model
    learners.append(Learner(pre_pipeline, model))


Preprocessing... {'missing': 'remove_rows', 'scaling': 'min_max', 'add_Solids_log': False, 'poly_degree': 3}
Preprocessing results: X of shape (2011, 228)
XGBClassifier(alpha=0, base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None, lambda=0,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=42, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)
[00:32:30] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' 

/home/nouamane/miniconda/envs/sklearn/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Preprocessing... {'missing': 'remove_rows', 'scaling': 'min_max', 'add_Solids_log': False, 'poly_degree': 3}
Preprocessing results: X of shape (2011, 228)
ExtraTreesClassifier(criterion='entropy', n_jobs=-1, random_state=42)

Preprocessing... {'missing': 'remove_rows', 'scaling': 'standard', 'add_Solids_log': True, 'poly_degree': 1}
Preprocessing results: X of shape (2011, 10)
KNeighborsClassifier(n_jobs=-1, n_neighbors=3, weights='distance')

Preprocessing... {'missing': 'remove_rows', 'scaling': 'standard', 'add_Solids_log': True, 'poly_degree': 3}
Preprocessing results: X of shape (2011, 295)
CalibratedClassifierCV(base_estimator=LinearSVC(max_iter=2000, random_state=42),
                       ensemble=False, method='isotonic', n_jobs=-1)


/home/nouamane/miniconda/envs/sklearn/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/nouamane/miniconda/envs/sklearn/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/nouamane/miniconda/envs/sklearn/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/nouamane/miniconda/envs/sklearn/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/nouamane/miniconda/envs/sklearn/lib/python3.9/site-pac


Preprocessing... {'missing': 'remove_rows', 'scaling': 'min_max', 'add_Solids_log': False, 'poly_degree': 2}
Preprocessing results: X of shape (2011, 63)
GaussianNB(var_smoothing=1e-05)


# Linear Regression Stacking

In [57]:
# remove SVC because it doesn't support predict_proba method
learners = [l for l in learners if not l.model.__str__().startswith('SVC')]

In [58]:
X = df.drop(["Potability", "kfold"], axis=1, errors='ignore').values
y = df.Potability.values

df_1 = pd.DataFrame()
X.shape

(3276, 9)

In [59]:
for i, (pre_pipeline, model) in enumerate(learners):
    X_processed, y_processed = pre_pipeline.transform(X, y) # TODO: case where we have different X_processed shapes
    try:
        pred = model.predict_proba(X_processed)
    except:
        print(f"error: {model.__class__}")
        continue
    
    print(pred.shape)
    df_1[f'pred{i}'] = pred[:,1]


(2011, 2)
(2011, 2)
(2011, 2)
(2011, 2)
(2011, 2)
(2011, 2)
(2011, 2)
(2011, 2)
(2011, 2)


In [60]:
df_1['Potability'] = y_processed
df_1

,pred0,pred1,pred2,pred3,pred4,pred5,pred6,pred7,pred8,Potability
0,0.019573,0.0,0.11,0.0,0.433044,0.0,0.0,0.301471,1.071604e-01,0
1,0.017951,0.0,0.20,0.0,0.096079,0.0,0.0,0.178261,9.993039e-01,0
2,0.007532,0.0,0.02,0.0,0.058016,0.0,0.0,0.178261,2.818165e-01,0
3,0.996133,1.0,0.92,1.0,0.894408,1.0,1.0,0.757009,9.128258e-01,1
4,0.003766,0.0,0.09,0.0,0.260188,0.0,0.0,0.279793,3.565109e-02,0
...,...,...,...,...,...,...,...,...,...,...
2006,0.018239,0.0,0.09,0.0,0.362581,0.0,0.0,0.389571,2.896901e-02,0
2007,0.006019,0.0,0.05,0.0,0.176581,0.0,0.0,0.279793,1.226180e-02,0
2008,0.000879,0.0,0.06,0.0,0.074214,0.0,0.0,0.178261,4.329454e-01,0
2009,0.018078,0.0,0.10,0.0,0.007563,0.0,0.0,0.178261,6.936564e-08,0


In [61]:
#TODO: the best models we trained always predict the same values
df_1.describe()

,pred0,pred1,pred2,pred3,pred4,pred5,pred6,pred7,pred8,Potability
count,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2.011000e+03,2011.000000
mean,0.403302,0.403282,0.405211,0.403282,0.398487,0.403282,0.403282,0.401509,3.736444e-01,0.403282
std,0.472497,0.490678,0.341537,0.490678,0.216725,0.490678,0.490678,0.177221,3.503922e-01,0.490678
min,0.000143,0.000000,0.000000,0.000000,0.000011,0.000000,0.000000,0.000000,5.537553e-10,0.000000
25%,0.010415,0.000000,0.110000,0.000000,0.255999,0.000000,0.000000,0.279793,6.991452e-02,0.000000
50%,0.026154,0.000000,0.190000,0.000000,0.370684,0.000000,0.000000,0.389571,2.285263e-01,0.000000
75%,0.974452,1.000000,0.790000,1.000000,0.503207,1.000000,1.000000,0.463415,6.749614e-01,1.000000
max,0.999985,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000


In [62]:
# create 5 folds
from sklearn.model_selection import StratifiedKFold
df_1.loc[:, "kfold"] = -1
kf = StratifiedKFold(n_splits=5)
for f, (t_, v_) in enumerate(kf.split(X=df_1, y=df_1['Potability'])):
    df_1.loc[v_, 'kfold'] = f
df_1

,pred0,pred1,pred2,pred3,pred4,pred5,pred6,pred7,pred8,Potability,kfold
0,0.019573,0.0,0.11,0.0,0.433044,0.0,0.0,0.301471,1.071604e-01,0,0
1,0.017951,0.0,0.20,0.0,0.096079,0.0,0.0,0.178261,9.993039e-01,0,0
2,0.007532,0.0,0.02,0.0,0.058016,0.0,0.0,0.178261,2.818165e-01,0,0
3,0.996133,1.0,0.92,1.0,0.894408,1.0,1.0,0.757009,9.128258e-01,1,0
4,0.003766,0.0,0.09,0.0,0.260188,0.0,0.0,0.279793,3.565109e-02,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2006,0.018239,0.0,0.09,0.0,0.362581,0.0,0.0,0.389571,2.896901e-02,0,4
2007,0.006019,0.0,0.05,0.0,0.176581,0.0,0.0,0.279793,1.226180e-02,0,4
2008,0.000879,0.0,0.06,0.0,0.074214,0.0,0.0,0.178261,4.329454e-01,0,4
2009,0.018078,0.0,0.10,0.0,0.007563,0.0,0.0,0.178261,6.936564e-08,0,4


In [63]:
# fit linear regression on preds
from sklearn.linear_model import LinearRegression
from src.evaluate import calculate_metrics

df = df_1
final_predictions = []
scores = []
metalearners = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df

    ytrain = xtrain.Potability
    yvalid = xvalid.Potability

    xtrain = xtrain.drop(["Potability", "kfold"], axis=1)
    xvalid = xvalid.drop(["Potability", "kfold"], axis=1)
    xtest = xtest.drop(["Potability", "kfold"], axis=1)
    
    model = LinearRegression()
    model.fit(xtrain, ytrain)
    print(xtrain.head())
    print(ytrain)
    metalearners.append(model)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    
    final_predictions.append(test_preds)
    score = calculate_metrics(yvalid, preds_valid)
    print(fold, score)
    scores.append(score)

      pred0  pred1  pred2  pred3     pred4  pred5  pred6     pred7     pred8
0  0.957351    1.0   0.86    1.0  0.717605    1.0    1.0  0.641026  0.350498
1  0.911250    1.0   0.73    1.0  0.283734    1.0    1.0  0.280000  0.268809
2  0.970488    1.0   0.83    1.0  0.420379    1.0    1.0  0.389571  0.073510
3  0.957190    1.0   0.73    1.0  0.414513    1.0    1.0  0.425676  0.191035
4  0.985783    1.0   0.89    1.0  0.463198    1.0    1.0  0.511628  0.113590
0       1
1       1
2       1
3       1
4       1
       ..
1603    0
1604    0
1605    0
1606    0
1607    0
Name: Potability, Length: 1608, dtype: int64
0 {'accuracy': 1.0, 'f1_score': 1.0, 'auc': 1.0}
      pred0  pred1  pred2  pred3     pred4  pred5  pred6     pred7     pred8
0  0.019573    0.0   0.11    0.0  0.433044    0.0    0.0  0.301471  0.107160
1  0.017951    0.0   0.20    0.0  0.096079    0.0    0.0  0.178261  0.999304
2  0.007532    0.0   0.02    0.0  0.058016    0.0    0.0  0.178261  0.281816
3  0.996133    1.0   0.92 

In [64]:
pred_probs = np.mean(np.column_stack(final_predictions), axis=1)
print(pred_probs.shape)
calculate_metrics(df_1.Potability.values, pred_probs)


(2011,)


{'accuracy': 1.0, 'f1_score': 1.0, 'auc': 1.0}

# Final Prediction Pipeline

In [85]:
# df = df.dropna()
df_orig = df.copy()

In [100]:
df = df_orig.copy()

In [79]:
X = df.drop(["Potability", "kfold"], axis=1, errors='ignore').values
y = df.Potability.values

In [80]:
preds1 = []
for pre_pipeline, model in learners:
    X_processed, _ = pre_pipeline.transform(X)
    preds1.append(model.predict_proba(X_processed)[:,1])
preds1 = np.column_stack(preds1)
preds1.shape

(2011, 9)

In [81]:
pd.DataFrame(preds1).describe()

,0,1,2,3,4,5,6,7,8
count,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2.011000e+03
mean,0.403302,0.403282,0.405211,0.403282,0.398487,0.403282,0.403282,0.401509,3.736444e-01
std,0.472497,0.490678,0.341537,0.490678,0.216725,0.490678,0.490678,0.177221,3.503922e-01
min,0.000143,0.000000,0.000000,0.000000,0.000011,0.000000,0.000000,0.000000,5.537553e-10
25%,0.010415,0.000000,0.110000,0.000000,0.255999,0.000000,0.000000,0.279793,6.991452e-02
50%,0.026154,0.000000,0.190000,0.000000,0.370684,0.000000,0.000000,0.389571,2.285263e-01
75%,0.974452,1.000000,0.790000,1.000000,0.503207,1.000000,1.000000,0.463415,6.749614e-01
max,0.999985,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00


In [88]:
final_preds = []
for model in metalearners:
    final_preds.append(model.predict(preds1))
final_preds = np.column_stack(final_preds)
print(final_preds.shape)
final_preds = np.mean(final_preds, axis=1)
final_preds = np.round(final_preds).astype(int)

(2011, 5)


In [89]:
pd.DataFrame(final_preds).describe()

,0
count,2011.000000
mean,0.403282
std,0.490678
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [90]:
df = df.dropna()
df['pred'] = final_preds
df

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability,pred
0,6.063355,160.767311,14775.14560,7.484104,305.828553,327.270239,12.309016,69.038454,3.467337,0,0
2,9.252857,168.040751,50279.26243,5.905056,415.450810,400.003589,11.949854,62.256881,3.300586,0,0
3,3.989032,216.076138,40175.20609,7.487423,309.800796,399.047181,11.186739,86.957464,4.409414,0,0
4,9.454119,224.817132,21379.96393,5.407692,227.665635,431.613001,15.772334,52.033845,4.058626,1,1
5,5.742533,182.941032,21293.88975,6.826413,306.543071,435.234727,11.144072,68.612391,4.272203,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3267,6.506488,202.285938,17087.07690,6.201983,326.662368,481.771934,16.535186,46.205748,3.548776,0,0
3268,4.965774,220.848639,21802.74490,6.250578,311.598136,386.413332,18.035238,53.908177,3.421767,0,0
3269,5.276324,186.382590,42063.87440,6.862988,319.837985,312.002533,18.964418,71.398976,3.883029,0,0
3270,13.349889,152.776455,18464.90077,6.717973,334.864070,450.846369,17.192564,85.883523,2.531075,0,0


In [91]:
# number of correct classifications
(df['pred'] == df['Potability']).sum()

2011

In [92]:
class BaggingClf():
    def __init__(self, learners, metalearners):
        self.learners = learners
        self.metalearners = metalearners
    def predict_proba(self, X: np.array):
        """ Predicts proba
        Arguments : X of shape (-1,9) 
        Returns : pred_probas of shape (-1,1) 
        """
        # Bootstrapping
        preds1 = []
        for pre_pipeline, model in self.learners:
            X_processed, _ = pre_pipeline.transform(X)
            preds1.append(model.predict_proba(X_processed)[:,1])
        preds1 = np.column_stack(preds1)

        # Aggregating
        final_preds = []
        for model in self.metalearners:
            final_preds.append(model.predict(preds1))
        final_preds = np.column_stack(final_preds)
        final_preds = np.mean(final_preds, axis=1)
        return final_preds

    def predict(self, X: np.array):
        """ Predicts class
        Arguments : X of shape (-1,9) 
        Returns : predicted_class of shape (-1,1) 
        """
        preds_probs = self.predict_proba(X)
        predicted_class = np.round(preds_probs).astype(int)
        return predicted_class

In [93]:
clf = BaggingClf(learners, metalearners)

In [101]:
X = df.drop(["Potability", "kfold"], axis=1, errors='ignore').values
y = df.Potability.values
preds_probs = clf.predict_proba(X)

In [103]:
calculate_metrics(df.dropna()['Potability'].values, preds_probs)

{'accuracy': 1.0, 'f1_score': 1.0, 'auc': 1.0}

In [104]:
from src.utils import save_file
from src import config, model_dispatcher
import pickle
model_name = 'bagging'
fold = -1

# save model
# save_file(clf, f"{config.SAVED_MODELS}/{model_name}/{model_name}_{fold}.bin")
with open(f"{config.SAVED_MODELS}/{model_name}/{model_name}_{fold}.bin", 'wb') as f:
    pickle.dump(clf, f)

In [105]:
class BaggingPrePipeline():
    def __init__(self):
        pass
    def transform(self, X):
        return X

# save preprocessing pipeline
save_file(
    BaggingPrePipeline, f"{config.SAVED_MODELS}/{model_name}/{model_name}_{fold}_preprocess.pkl")